In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
pd.options.display.max_columns = None

In [ ]:
data = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.drop('customerID', axis = 1, inplace = True)

In [ ]:
set(''.join(data['TotalCharges'].tolist()))

In [ ]:
data['TotalCharges'] = data['TotalCharges'].replace(' ', np.nan)

In [ ]:
data['TotalCharges'] = data['TotalCharges'].astype('float')

In [ ]:
data.info()

In [ ]:
data.columns = data.columns.str.lower()

In [ ]:
data.columns

In [ ]:
num_cols = ['tenure','monthlycharges', 'totalcharges']

In [ ]:
data[num_cols].describe().T

In [ ]:
data[[col for col in data.columns.difference(num_cols) if col != 'seniorcitizen']].describe().T

In [ ]:
data['seniorcitizen'].value_counts()

In [ ]:
ord_cols = ['dependents', 'gender', 'paperlessbilling', 'partner', 'phoneservice']
label = 'churn'
cat_cols = ['seniorcitizen', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract','paymentmethod']

In [ ]:
plt.figure(figsize = (15, 4))
for i, col in enumerate(num_cols):
    plt.subplot(1, 3, i+1)
    sns.histplot(data, x= col, color = 'red', alpha = 0.2, kde = True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (15, 4))
for i, col in enumerate(num_cols):
    plt.subplot(1, 3, i+1)
    sns.rugplot(data, x = col, hue= label, height = 0.1)
    sns.boxplot(data, x = col, width = 0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (15, 4))
for i, col in enumerate(num_cols):
    plt.subplot(1, 3, i+1)
    sns.boxplot(data, x = label, y = col, width = 0.4)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (15, 26))
for i, col in enumerate(data.columns.difference(num_cols)[1:]):
    plt.subplot(6, 3, i+1)
    ax = sns.countplot(data, x = col, hue = label)
    ax.bar_label(ax.containers[0])
    ax.bar_label(ax.containers[1])
    plt.xticks(rotation = 15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (4,4))
ax = sns.countplot(data, x = label)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
def lencoder(col):
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    return data[col]

In [ ]:
for col in ord_cols:
    data[col] = lencoder(col)

In [ ]:
data['churn'] = lencoder('churn')

In [ ]:
ohe_data = pd.get_dummies(data)

In [ ]:
ohe_data[ohe_data.select_dtypes(include = 'bool').columns] = ohe_data[ohe_data.select_dtypes(include = 'bool').columns].astype('int')

In [ ]:
data = ohe_data.copy()

In [ ]:
imp_mean = IterativeImputer(random_state = 42)

In [ ]:
data_impute = imp_mean.fit_transform(data)

In [ ]:
data = pd.DataFrame(data_impute, columns = data.columns)

In [ ]:
!pip -q install pycaret

In [ ]:
!pip -q install --upgrade scipy

In [ ]:
!pip -q install --upgrade yellowbrick

In [ ]:
import pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
s = setup(data, target = 'churn', session_id = 42, data_split_stratify=True)

In [ ]:
best_model = compare_models(sort = 'AUC')

In [ ]:
print(best_model)

In [ ]:
plt.figure(figsize = (7, 4))
plot_model(best_model, plot = 'feature')

In [ ]:
plt.figure(figsize = (7, 4))
plot_model(best_model, plot = 'auc')

In [ ]:
plt.figure(figsize = (4,3))
plot_model(best_model, plot = 'confusion_matrix')

In [ ]:
def calculate_profit(y, y_pred):
    tp = np.where((y_pred == 1) & (y == 1), 4000, 0)
    fp = np.where((y_pred == 1) & (y == 0), -1000, 0)
    return np.sum([tp,fp])

In [ ]:
add_metric('profit', 'Profit', calculate_profit)

In [ ]:
best_model = compare_models(sort = 'Profit')

In [ ]:
plt.figure(figsize = (4,3))
plot_model(best_model, plot = 'confusion_matrix')

In [ ]:
save_model(best_model, 'churn-predict')